(pratica_isostasia)=
# Prática 2: Isostasia, flexura e gravidade

```{admonition} Objetivos
:class: important

1. Observar como a teoria da compensação isostática se reflete nos dados observados.
1. Aprender as técnicas práticas que são necessárias para conectar a teoria à modelagem dos dados.
1. Explorar o estado isostático de diferentes regiões do planeta.
1. Obter uma ferramenta que nos permita investigar a rigidez da placa oceânica através de dados de gravidade.
```

```{admonition} Antes de começar
:class: seealso

Esta prática depende do conteúdo das aulas:

* {ref}`litosfera`
* {ref}`gravidade`
* {ref}`isostasia`
* {ref}`pratica_grav`
```

## Bibliotecas

Vamos primeiro carregar as bibliotecas que vamos utilizar nessa prática.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import xrft
import harmonica as hm
import verde as vd
import pyproj
import pygmt
import ipywidgets
import warnings

## Dados

Nesta prática, vamos utilizar os dados de distúrbio da gravidade, anomalia Bouguer simples e topografia que compilamos na prática passada.

## Bouguer vs topografia

Como vimos na aula teórica, nos casos em que há compensação isostática segundo o modelo de Airy esperamos que a anomalia Bouguer possua uma **relação linear** com a topografia. Vamos verificar se isso é verdade para a Terra como um todo utilizando nossos dados.

A princípio parece que sim! Vamos calcular o valor teórico predito para ser se nosso modelo se ajusta aos dados. Lembrando que:

* Nos **continentes**: $ \delta g_{bg} = - 2 \pi G \rho_c h $
* Nos **oceanos**: $ \delta g_{bg} = - 2 \pi G (\rho_c - \rho_w) h $
* Constante gravitacional: $G = 6.67430 \times 10^{-11}\ m^3 kg^{-1} s^{-2}$

In [ ]:
def bouguer_airy(topografia, densidade_crosta_continental, densidade_crosta_oceanica, densidade_agua):
    """
    Calcula a anomalia Bouguer prevista por um modelo de Airy.
    """
    G = 6.67430e-11
    bouguer = 
    
    return bouguer

Com essa função, podemos prever os dados de anomalia Bouguer utilizando densidades médias para a crosta. 

:::{admonition} 💬 **Discussão**
:class: tip

Com base na figura acima 👆🏽, discuta em grupos de 2-3:

1. Quais ambientes tectônicos são responsáveis pelo desajuste no canto superior esquerdo e inferior direito da figura? 
1. Qual é uma possível explicação geofísica/tectônica para esse desajuste?

:::

::::{admonition} 🧘 **Sua vez: O que acontece nas dorsais?** 
:class: tip

As dorsais estão em equilíbrio isostático, como vimos no mapa do distúrbio da gravidade na aula passada. Porém, o modelo Airy não é adequado para explicar a compensação isostática das dorsais.

1. Qual modelo de compensação explica o equilíbrio nas dorsais?
1. Derive a relação entre topografia e anomalia Bouguer para esse modelo e a implemente em uma função.
1. Selecione uma parcela dos dados que contém uma dorsal e aplique seu modelo.
1. O modelo ajusta os dados? Como esse modelo difere da previsão utilizando Airy?
1. Qual é uma possível interpretação geológica/geofísica qu justifica o uso do modelo Pratt?

::::

## Ilhas oceânicas

Uma das regiões que podemos identificar pelo mapa do distúrbio da gravidade como não estando compensadas isostaticamente são as ilhas oceânicas como o Havaí. 

In [ ]:
plt.figure(figsize=(12, 7))
dados.disturbio.plot()

Vamos isolar os dados em torno o arquipélago Havaiano para ter uma ideia melhor dos valores de topografia e distúrbio da gravidade.

Podemos verificar pelo gráfico de anamolia Bouguer por topografia que essa região não está compensada por um mecanismo Airy ou Pratt.

## Flexura da litosfera oceânica

A topografia elevada das ilhas oceânicas é sustentada pela própria rigidez da placa oceânica, que se deforma sob a carga das ilhas. Como vimos na aula teórica, nós podemos modelar essa deformação (a flexura) através da solução de uma equação diferencial utilizando a transformada rápida de Fourier (FFT).

A aplicação da FFT necessita dados em coordenadas cartesianas. Logo, primeiro devemos projetar nossos dados. Aqui utilizarmos uma projeção Mercator. No final, teremos nossos dados com coordenadas Leste e Norte em metros.

Em seguida, precisamos calcular somente o desvio da topografia com relação a uma batimetria mediana. Isso é necessário para alinhar nossos dados com o que o nosso modelo matemático espera.

Uma feição comum em ilhas oceânicas associadas com *hotspots* é um inchaço da crosta ao redor das ilhas (**thermal swell**) que é atribuído à dilatação térmica da litosfera causada pelo hotspot. Isso não faz parte do nosso modelo de flexura e então precisamos removê-lo antes da nossa modelagem. Faremos isso com um **filtro passa-alta** para remover esse efeito de longo comprimento de onda.

Agora podemos calcular o filtro no domínio da frequência entre a topografia e a flexura:

$$
W(k_x, k_y) = \dfrac{-(\rho_c - \rho_w)}{(\rho_m - \rho_c)}\left[1 + \dfrac{D(2\pi k)^4}{g(\rho_m - \rho_c)}\right]^{-1} T(k_x, k_y)
$$

$$k = \sqrt{k_x^2 + k_y^2}$$

In [ ]:
def filtro_flexura(
    kx, ky, Te, densidade_crosta=2800, densidade_agua=1000, densidade_manto=3300, 
    young=6.5e10, poisson=0.25, gravidade=9.8,
):
    """
    Calcula o filtro da flexura no domínio da frequência.
    """
    D = young * Te ** 3 / (
        12 * (1 - poisson ** 2)
    )
    k = np.sqrt(ky**2 + kx**2)
    filtro = (
        
    )
    return filtro

Agora podemos aplicar a FFT aos dados de topografia para realizar a modelagem. 

::::{admonition} 🧘 **Sua vez: A espessura elástica efetiva** 
:class: tip

A espessura elástica efetiva controla a forma da flexura da litosfera oceânica. Utilizando a figura interativa acima, responda:

1. Qual é o efeito que aumentar ou diminuir $T_e$ tem na flexura estimada?
1. Quando $T_e = 0$, o que acontece com nosso modelo de flexura? Observe tanto o resultado acima quanto a equação do filtro de flexura acima.
1. Dada a questão anterior, qual relação você espera que exista entre $T_e$ e o distúrbio da gravidade sobre as ilhas?

::::

## Distúrbio da gravidade versus topografia

Vimos também em aula que podemos calcular o distúrbio da gravidade causado pela flexura assumindo que a Moho é deformada da mesma forma que a batimetria. Como temos dados de gravidade, podemos aplicar a FFT aos dados do distúrbio e dividir pela FFT da topografia. Essa operação estima o **valor do filtro entre a topografia e o distúrbio**, que podemos modelar com a equação:

$$
\phi(k_x, k_y) = 2 \pi G (\rho_c - \rho_w) e^{-2\pi k s} \left\{ 1 -  \left[1 + \dfrac{D(2\pi k)^4}{g(\rho_m - \rho_c)}\right]^{-1}e^{-2\pi k d} \right\}
$$

Mas antes disso, precisamos remover o efeito do *swell* dos nossos dados de distúrbio da gravidade, como fizemos para a topografia.

Agora podemos aplicar a FFT e estimar o filtro observado como função do número de onda.

Nos altos números de onda, o filtro apresenta um comportamento disperso. Isso pode estar associado às diversas **zonas de falhas transformantes** que estão presentes na região e não obedecem a relação da flexura.

Vamos calcular o valor predito do filtro e comparar com as observações.

In [ ]:
def filtro_flexura_grav(
    kx, ky, Te, distancia, espessura_crosta,
    densidade_crosta=2800, densidade_agua=1000, densidade_manto=3300, 
    young=6.5e10, poisson=0.25, gravidade=9.8,
):
    """
    Calcula o filtro para prever grav causado pela flexura.
    """
    G = 6.67430e-11
    D = young * Te ** 3 / (
        12 * (1 - poisson ** 2)
    )
    k = np.sqrt(ky**2 + kx**2)
    filtro = (
      
    
    )
    return filtro

Como podemos ver, o filtro observado pode ser utilizado para **estimar a espessura elástica efetiva** da litosfera oceânica.